In [ ]:
#Copy Paste Sub routine

import cv2
# Inspired from https://www.mdpi.com/2072-4292/12/16/2558/pdf

# Create a class object for efficiency
class copyPaste():
  def __init__(self, annotation):
    self.ann = copy.deepcopy(annotation)

  def load_image(self, image = None):
    self.image = image
    if self.image is None:
      self.image = utils.read_image(self.ann["file_name"], format="BGR")
    self.mask()

  def mask(self):
    # TODO add more accurate sea-land masking or bathymetery data
    #https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_thresholding/py_thresholding.html
    FILTER = 3
    BLUR = 5
    temp = cv2.imread(self.ann["file_name"], 0)
    temp = cv2.equalizeHist(temp)
    temp = cv2.GaussianBlur(temp,(BLUR,BLUR),0)
    ret,th = cv2.threshold(temp,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    mask = np.where(th == 255, 1, 0).astype('uint8')
    kernel = np.ones((FILTER,FILTER),np.uint8)
    self.mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

  def valid_copy(self, xmin, xmax, ymin, ymax):
    valid = xmin > 0 and xmax < 800 and ymin > 0 and ymax < 800
    bboxs = [i['bbox'] for i in self.ann['annotations']]
    iou = self.calc_iou(bboxs, [xmin, ymin, xmax, ymax])
    valid = valid and np.all(np.isclose(iou, 0))
    valid = valid and (0 == np.sum(self.mask[ymin:ymax, xmin:xmax]))
    return valid 

  def calc_iou(self, bboxes, bbox):
    #https://medium.com/@venuktan/vectorized-intersection-over-union-iou-in-numpy-and-tensor-flow-4fa16231b63d
    x11, y11, x12, y12 = np.split(np.array(bboxes), 4, axis=1)
    x21, y21, x22, y22 = np.split(np.array(bbox), 4, axis=0)
    xA = np.maximum(x11, np.transpose(x21))
    yA = np.maximum(y11, np.transpose(y21))
    xB = np.minimum(x12, np.transpose(x22))
    yB = np.minimum(y12, np.transpose(y22))
    interArea = np.maximum((xB - xA + 1), 0) * np.maximum((yB - yA + 1), 0)
    boxAArea = (x12 - x11 + 1) * (y12 - y11 + 1)
    boxBArea = (x22 - x21 + 1) * (y22 - y21 + 1)
    iou = interArea / (boxAArea + np.transpose(boxBArea) - interArea)
    return iou

  def parse(self):
    bbox = self.ann['annotations'][self.index]['bbox']
    self.xmin = bbox[0]
    self.xmax = bbox[2]
    self.ymin = bbox[1]
    self.ymax = bbox[3]

  def copyImg(self):
    self.parse()
    self.copy = self.image[self.ymin: self.ymax, 
                 self.xmin:self.xmax, :]
    
  def translate(self):
    xmin = self.xmin + self.xdel
    xmax = self.xmax + self.xdel
    ymin = self.ymin + self.ydel
    ymax = self.ymax + self.ydel
    return (xmin, xmax, ymin, ymax)

  def update_ann(self, xmin, ymin, xmax, ymax):
    new = {'bbox': [xmin, ymin, xmax, ymax],
           'bbox_mode': BoxMode.XYXY_ABS,
           'category_id': 0}
    self.ann['annotations'].append(new)

  def copyPaste(self, index, xdel, ydel):
    self.index = index
    self.copyImg()
    self.xdel = xdel
    self.ydel = ydel
    xmin, xmax, ymin, ymax = self.translate()
    if self.valid_copy(xmin, xmax, ymin, ymax):
      temp = self.image.copy()
      temp[ymin:ymax, xmin:xmax, :] = self.copy
      self.image = temp
      self.update_ann(xmin, ymin, xmax, ymax)

  def getImage(self):
    return self.image
  
  def getAnn(self):
    return self.ann